# Detect Entities and Link them to Wikipedia and Wikidata in a Text through the Impresso API

We refer to "named entity recognition" as NER, which is a tool that recognises entities such as persons and locations from text. A "named entity linker" (NEL) connects these entities to an existing one such as a real person that can be found on Wikipedia (with a unique id in Wikidata). Wikipedia is a free, user-edited encyclopedia with articles on a wide range of topics like historical events, famous people, or scientific concepts. Wikidata is a sister project of Wikipedia that stores structured data, like facts and relationships between entities, used for tasks where computers need to understand and process data, such as NER and NEL.


In the context of _Impresso_, the NER tool was trained on the [HIPE 2020](https://github.com/hipe-eval/HIPE-2022-data/blob/main/documentation/README-hipe2020.md) dataset. It was trained to recognise coarse and fine grained entities such as persons and locations, but also their names, titles, and functions. Further, the _Impresso_ NEL tool links these entity mentions to unique referents in a knowledge base – here Wikipedia and Wikidata – or not if the mention's referent is not found.

In [ ]:
!pip install transformers
!wget https://raw.githubusercontent.com/impresso/impresso-datalab-notebooks/3f7afc05caef3f527db8320cdf8c131aec41d7cd/2-entity/utils.py


--2024-09-30 12:30:21--  https://raw.githubusercontent.com/impresso/impresso-datalab-notebooks/3f7afc05caef3f527db8320cdf8c131aec41d7cd/2-entity/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5625 (5.5K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   5.49K  --.-KB/s    in 0s      

2024-09-30 12:30:21 (62.4 MB/s) - ‘utils.py’ saved [5625/5625]



In [7]:
def print_nicely(results, text):
    # Print the timestamp and system ID
    print(f"Timestamp: {results.get('ts')}")
    print(f"System ID: {results.get('sys_id')}")

    entities = results.get('nes', [])
    if entities:
        print(f"\n{'Entity':<20} {'Type':<15} {'Confidence NER':<15} {'Confidence NEL':<15} {'Start':<5} {'End':<5} {'Wikidata ID':<10} {'Wikipedia Page':<20}")
        print("-" * 100)
        for entity in entities:
            confidence_ner = f"{entity['confidence_ner']}%"
            confidence_nel = f"{entity['confidence_nel']}%"
            wkd_id = entity.get('wkd_id', 'N/A')
            wkpedia_pagename = entity.get('wkpedia_pagename', 'N/A')
            print(f"{entity['surface']:<20} {entity['type']:<15} {confidence_ner:<15} {confidence_nel:<15} {entity['lOffset']:<5} {entity['rOffset']:<5} {wkd_id:<10} {wkpedia_pagename:<20}")

        print("*" * 100)
        print('Testing offsets:')
        print("*" * 100)
        print(f"\n{'Entity':<20} {'Type':<15} {'Confidence NER':<15} {'Confidence NEL':<15} {'Start':<5} {'End':<5} {'Wikidata ID':<10} {'Wikipedia Page':<20}")
        print("-" * 100)
        for entity in entities:
            confidence_ner = f"{entity['confidence_ner']}%"
            confidence_nel = f"{entity['confidence_nel']}%"
            wkd_id = entity.get('wkd_id', 'N/A')
            wkpedia_pagename = entity.get('wkpedia_pagename', 'N/A')
            print(f"{text[entity['lOffset']:entity['rOffset']]:<20} {entity['type']:<15} {confidence_ner:<15} {confidence_nel:<15} {entity['lOffset']:<5} {entity['rOffset']:<5} {wkd_id:<10} {wkpedia_pagename:<20}")

        print("*" * 100)
        print('Testing offsets in the returned text:')
        print("*" * 100)
        print(f"\n{'Entity':<20} {'Type':<15} {'Confidence NER':<15} {'Confidence NEL':<15} {'Start':<5} {'End':<5} {'Wikidata ID':<10} {'Wikipedia Page':<20}")
        print("-" * 100)
        for entity in entities:
            confidence_ner = f"{entity['confidence_ner']}%"
            confidence_nel = f"{entity['confidence_nel']}%"
            wkd_id = entity.get('wkd_id', 'N/A')
            wkpedia_pagename = entity.get('wkpedia_pagename', 'N/A')
            print(f"{results['text'][entity['lOffset']:entity['rOffset']]:<20} {entity['type']:<15} {confidence_ner:<15} {confidence_nel:<15} {entity['lOffset']:<5} {entity['rOffset']:<5} {wkd_id:<10} {wkpedia_pagename:<20}")



Now the fun part, this function will download the requried model and gives you the keys to successfullly detect entities in your text.

In [8]:
sentences = ["Apple est créée le 1er avril 1976 dans le garage de la maison d'enfance de Steve Jobs à Los Altos en Californie par Steve Jobs, Steve Wozniak et Ronald Wayne, puis constituée sous forme de société le 3 janvier 1977 à l'origine sous le nom d'Apple Computer, mais pour ses 30 ans et pour refléter la diversification de ses produits, le mot « computer » est retiré le 9 janvier 2015."]

for sentence in sentences:
    results = get_linked_entities(sentence)
    print(results)
    print_nicely(results, sentence)


{'ts': '2024-09-30T15:31:45Z', 'sys_id': 'ner-mgenre-multilingual', 'nes': [{'surface': 'Apple', 'lOffset': 0, 'rOffset': 17, 'type': 'UNK', 'id': '0:17:Apple:ner-mgenre-multilingual', 'wkd_id': 'Q312', 'wkpedia_pagename': 'Apple >> fr ', 'confidence_nel': 100.0}, {'surface': '', 'lOffset': 0, 'rOffset': 24, 'type': 'UNK', 'id': '0:24::ner-mgenre-multilingual', 'wkd_id': 'Q312', 'wkpedia_pagename': 'Apple >> fr ', 'confidence_nel': 30.57}, {'surface': '', 'lOffset': 0, 'rOffset': 24, 'type': 'UNK', 'id': '0:24::ner-mgenre-multilingual', 'wkd_id': 'Q312', 'wkpedia_pagename': 'Apple >> fr ', 'confidence_nel': 99.96}, {'surface': '', 'lOffset': 0, 'rOffset': 24, 'type': 'UNK', 'id': '0:24::ner-mgenre-multilingual', 'wkd_id': 'Q312', 'wkpedia_pagename': 'Apple >> fr ', 'confidence_nel': 34.13}, {'surface': '', 'lOffset': 0, 'rOffset': 24, 'type': 'UNK', 'id': '0:24::ner-mgenre-multilingual', 'wkd_id': 'Q312', 'wkpedia_pagename': 'Apple >> fr ', 'confidence_nel': 99.96}, {'surface': '', 'lO

KeyError: 'confidence_ner'


## About Impresso

### Impresso project

[Impresso - Media Monitoring of the Past](https://impresso-project.ch) is an
interdisciplinary research project that aims to develop and consolidate tools for
processing and exploring large collections of media archives across modalities, time,
languages and national borders. The first project (2017-2021) was funded by the Swiss
National Science Foundation under grant
No. [CRSII5_173719](http://p3.snf.ch/project-173719) and the second project (2023-2027)
by the SNSF under grant No. [CRSII5_213585](https://data.snf.ch/grants/grant/213585)
and the Luxembourg National Research Fund under grant No. 17498891.

### Copyright

Copyright (C) 2024 The Impresso team.

### License

This program is provided as open source under
the [GNU Affero General Public License](https://github.com/impresso/impresso-pyindexation/blob/master/LICENSE)
v3 or later.

---

<p align="center">
  <img src="https://github.com/impresso/impresso.github.io/blob/master/assets/images/3x1--Yellow-Impresso-Black-on-White--transparent.png?raw=true" width="350" alt="Impresso Project Logo"/>
</p>
